<a href="https://colab.research.google.com/github/BehnoodRasti/SparseEDAA/blob/main/SparseEDAA_Alex_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#!git clone https://github.com/BehnoodRasti/Unmixing_Tutorial_IEEE_IADF.git
#!git clone https://github.com/BehnoodRasti/SUnCNN.git

In [24]:
#!pip install munkres

In [25]:
import time
import os

import matplotlib.pyplot as plt
import scipy.io as sio
import numpy as np
import numpy.linalg as LA
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm as tqdm
from tqdm import tqdm

from munkres import Munkres
from Unmixing_Tutorial_IEEE_IADF.utils.data import HSI
from Unmixing_Tutorial_IEEE_IADF.utils.metrics import MeanSquareError as MSE
from Unmixing_Tutorial_IEEE_IADF.utils.metrics import aRMSE as RMSE
from Unmixing_Tutorial_IEEE_IADF.utils.metrics import SADDegrees as SAD 
from Unmixing_Tutorial_IEEE_IADF.utils.aligners import MunkresAbundancesAligner as Aligner
from IPython.display import clear_output
import sys

In [26]:
import scipy.io
import numpy as np
fname2  = "Data/DC1/Y_clean.mat"
mat2 = scipy.io.loadmat(fname2)
img_np_gt = mat2["Y_clean"]
img_np_gt = img_np_gt.transpose(2,0,1)
[p1, nr1, nc1] = img_np_gt.shape
img_resh=np.reshape(img_np_gt,(p1,nr1*nc1))
#%%
fname3  = "Data/DC1/XT.mat"
mat3 = scipy.io.loadmat(fname3)
A_true_np = mat3["XT"]

#%%
fname4  = "Data/DC1/EE.mat"
mat4 = scipy.io.loadmat(fname4)
EE = mat4["EE"]
D=EE
#%%
p = 5 # number of endmembers

In [27]:
#LA.norm(img_resh, axis=0, ord=2, keepdims=True)

In [28]:
Y = img_resh

In [29]:
L, N = Y.shape

In [30]:
# indices = np.random.choice(np.arange(N), size=100)

In [31]:
# plt.plot(img_resh[:, indices]);

In [32]:
# Y = Y / LA.norm(Y, axis=0, ord=2, keepdims=True)
# plt.plot(Y[:, indices]);

In [33]:
# !pip install cvxpylayers

In [34]:
# import cvxpy as cp
# import torch
# from cvxpylayers.torch import CvxpyLayer

# p, N = 4, 100
# LibS=220
# x = cp.Variable((p,N))
# AA = cp.Parameter((LibS, p))
# bb = cp.Parameter((LibS,N))
# onesp=torch.ones((p,1))#.to(self.device) 
# onesn=torch.ones((N,1))#.to(self.device) 
# constraints = [onesp.t()@x == onesn.t()]
# objective = cp.Minimize(0.5 * cp.pnorm((AA @ x - bb)**2, p=2))
# problem = cp.Problem(objective, constraints)
# assert problem.is_dpp()

# cvxpylayer = CvxpyLayer(problem, parameters=[AA, bb], variables=[x])
# A_tch = torch.randn(LibS, p, requires_grad=True)
# b_tch = torch.randn(LibS,N, requires_grad=True)

# # solve the problem
# solution, = cvxpylayer(A_tch, b_tch,'ignore_dpp = True')
# print(solution)
# # compute the gradient of the sum of the solution with respect to A, b
# solution.sum().backward()

In [39]:
class SparseEDAA:
    def __init__(self,lamb=0.1, mu1=20,mu2=10,mu3=1, TA=5, TB=10,T=20000):
        self.mu1 = mu1
        self.mu2 = mu2
        self.mu3 = mu3
        self.TA = TA
        self.TB = TB
        self.T =T
        self.lamb=lamb
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    def solve(
        self,
        Y,
        D,
        p,
        seed=0,
        **kwargs,
    ):
        
        L, N = Y.shape
        LibS=D.shape[1]
        def residual(a, b):
            return 0.5 * ((Y - (D @ b) @ a) ** 2).sum()

        def lossTot(a, b, lamb):
            return residual(a, b) + lamb * b.abs().sum()
        results = {}
        #eps=sys.float_info.epsilon
        tic = time.time()

        #for m in tqdm(range(self.M)):
        #    torch.manual_seed(m + seed)
        #    generator = np.random.RandomState(m + seed)
        #    with torch.no_grad():
        A = (1 / p) * torch.ones((p, N))
        B=(1 / LibS) * torch.ones((LibS, p))
        L1 = torch.zeros((p,N))
        L2 = torch.zeros((LibS,p))
        L3 = torch.zeros((L,p))
        S1=L1
        S2=L2
        S3=L3
        inv1=torch.zeros((p,p))
        inv2=torch.zeros((1,1))
        inv3=torch.zeros((LibS,LibS))
        inv4=inv2
        # Send matrices on GPU
        D = D.to(self.device)
        Y = Y.to(self.device)
        A = A.to(self.device)
        B = B.to(self.device)
        S1 = S1.to(self.device)
        S2 = S2.to(self.device)
        S3 = S3.to(self.device)
        L1 = L1.to(self.device)
        L2 = L2.to(self.device)
        L3 = L3.to(self.device)
        eyep=torch.eye(p).to(self.device) 
        eyeLibS=torch.eye(LibS).to(self.device) 
        onesp=torch.ones((p,1)).to(self.device) 
        onesLibS=torch.ones((LibS,1)).to(self.device) 
        onesn=torch.ones((N,1)).to(self.device) 
        eyeL=torch.eye(L).to(self.device) 
        # Random Step size factor
        DtY=D.t()@Y
        inv3=torch.inverse(self.mu3*D.t()@D+self.mu2*eyeLibS)#+eyeLibS*eps)
        inv4=-1/(onesLibS.t()@inv3@onesLibS)
        inv5=torch.inverse(self.mu2*eyeLibS+self.mu3*D.t()@D)
        soft=torch.nn.Softshrink(self.lamb/self.mu2)
        hard=torch.nn.Hardshrink(np.sqrt(2*self.lamb/self.mu2))
        for ii in tqdm(range(self.T)):
            for jj in range(self.TA):
                inv1=torch.inverse(self.mu3*S3.t()@S3+self.mu1*eyep)
                inv2=-1/(onesp.t()@inv1@onesp)
                A=(inv1+inv1@onesp@inv2@onesp.t()@inv1)@(S3.t()@Y+self.mu1*(S1-L1))-inv1@onesp@(inv2*onesn.t())
                S1= (A+L1)
                S1[S1<=0]=0
                L1 = L1 + A - S1
                #print(lossTot(A,B,self.lamb).item())
            for jj in range(self.TB):
                #B=(inv3+inv3@onesLibS@inv4@onesLibS.t()@inv3)@(mu3*D.t()@(S3-L3)+mu2*(S2-L2))-inv3@onesLibS@(inv4*onesp.t())
                B=inv5@(self.mu2*(S2-L2)+self.mu3*D.t()@(S3-L3))
                #S2= (B+L2)
                #============
                S2= hard(B+L2)
                S2[S2<=0]=0
                #===========
                S2[S2>=1]=1
                #=========== 
                L2 = L2 + B - S2
                S3=(Y@A.t() + self.mu3*(D@B+L3))@torch.inverse(A@A.t()+self.mu3*eyep)
                L3 = L3 + D@B - S3
                print(lossTot(A,B, self.lamb).item())
                # inv1=torch.inverse(self.mu3*S3.t()@S3+self.mu1*eyep)
                # inv2=-1/(onesp.t()@inv1@onesp)
                # A=(inv1+inv1@onesp@inv2@onesp.t()@inv1)@(S3.t()@Y+self.mu1*(S1-L1))-inv1@onesp@(inv2*onesn.t())
                # B=(inv3+inv3@onesLibS@inv4@onesLibS.t()@inv3)@(self.mu3*D.t()@(S3-L3)+self.mu2*(S2-L2))-inv3@onesLibS@(inv4*onesp.t())
                # S3=(Y@A.t() + self.mu3*(D@B+L3))@torch.inverse(A@A.t()+self.mu3*eyep)
                # S1= (A+L1)
                # S1[S1<=0]=0
                # S2= (B+L2)
                # S2[S2<=0]=0
                # L1 = L1 + A - S1
                # L2 = L2 + B - S2
                # L3 = L3 + D@B - S3
            #print(lossTot(A,B,self.lamb).item())
            if ii % 10 == 0:
                    clear_output(wait=False)
        E = (D @ B).cpu().numpy()
        A = A.cpu().numpy()
        B = B.cpu().numpy()
        # Store results
        results[1] = {
            "Em": E,
            "Am": A,
            "Bm": B,
        }

        toc = time.time()
        elapsed_time = round(toc - tic, 2)
        print(f"{self} took {elapsed_time}s")

        return E,B, A

    def __repr__(self):
        msg = f"{self.__class__.__name__}"
        return msg

In [40]:
import numpy as np


class AdditiveWhiteGaussianNoise:
    def __init__(self):
        self.sigmas = None
        self.SNR = None
        self.L = None
        self.N = None

    def fit(self, Y, SNR: float):
        """
        Compute sigmas at the desired SNR given a flattened input HSI Y
        """
        assert len(Y.shape) == 2
        self.L, self.N = Y.shape
        #logger.debug(f"Y shape: {Y.shape}")
        self.SNR = SNR
        #logger.debug(f"Desired SNR: {self.SNR}")

        if SNR is None:
            self.sigmas = np.zeros(self.L)
        else:
            assert SNR > 0, "SNR must be stricly positive"
            # Uniform across bands
            sigmas = np.ones(self.L)
            # Normalization
            sigmas /= np.linalg.norm(sigmas)
            #logger.debug(f"Sigmas after normalization: {sigmas[0]}")
            # compute mean sigma
            num = np.sum(Y ** 2) / self.N
            denom = 10 ** (self.SNR / 10)
            sigmas_mean = np.sqrt(num / denom)
            #logger.debug(f"Sigma mean based on SNR: {sigmas_mean}")
            # Noise variance
            sigmas *= sigmas_mean
            #logger.debug(f"Final sigmas value: {sigmas[0]}")
            self.sigmas = sigmas

    def transform(self, Y, seed=0):
        """
        Add White Gaussian Noise to the flattened input HSI Y
        """
        assert self.sigmas is not None, "Must be fitted first"
        assert (self.L, self.N) == Y.shape
        # Fix random seed
        generator = np.random.RandomState(seed=seed)
        # Noise generation
        N = np.diag(self.sigmas) @ generator.randn(self.L, self.N)
        # Additive Noise
        return Y + N

    def fit_transform(self, Y, SNR, seed=0):
        """
        Combine `fit` and `transform` methods.
        See above for their respective usage.
        """
        self.fit(Y, SNR)
        return self.transform(Y, seed=seed)

In [41]:
# # Subspace Setup
# from scipy import linalg
# model = SparseEDAA(K1=1, K2=1, T=20000, M=1)
# #Y = img_resh
# D=EE
# #img_mean=np.mean(img_resh,1).reshape(p1,1)
# #img_mean_rmv=img_resh-img_mean
# #V, SS, U = scipy.linalg.svd(img_mean_rmv@img_mean_rmv.transpose(1,0)/(nr1*nc1), full_matrices=False)
# #Y=V[:,:p].transpose(1,0)@img_mean_rmv
# Y = noise.fit_transform(img_resh, SNR=40, seed=0)
# V, SS, U = scipy.linalg.svd(Y, full_matrices=False)
# Y_r=V[:,:p].transpose(1,0)@Y
# DD = V[:,:p].transpose(1,0)@D
# noise = AdditiveWhiteGaussianNoise()
# Y_r = torch.Tensor(Y_r)
# DD = torch.Tensor(DD)
# D = torch.Tensor(D)
# E00, A0 = model.solve(Y_r, DD, p, seed=1)
# E0=V[:,:p]@E00

In [44]:
# Instantiate objects
model = SparseEDAA(lamb=1, mu1=1000,mu2=100,mu3=1, TA=10,TB=10,T=10000)
Y = img_resh
D = EE#np.concatenate((EE, Y), axis=1)
noise = AdditiveWhiteGaussianNoise()
Y = noise.fit_transform(Y, SNR=20, seed=0)
from scipy import linalg
V, SS, U = scipy.linalg.svd(Y, full_matrices=False)
PC=np.diag(SS)@U
Y_DN=V[:,:p]@PC[:p,:]
# Convert the data to tensors
Y = torch.Tensor(Y)
Y_DN = torch.Tensor(Y_DN)
D = torch.Tensor(D)
# Abundances estimation
E0,B0, A0 = model.solve(Y_DN, D, p, seed=1)

112.31419372558594
150.33242797851562
187.831787109375
219.87841796875
242.65097045898438
253.76405334472656
252.43963623046875
239.44412231445312
216.85064697265625
187.6556854248047
155.32473754882812
123.3783187866211
94.88697814941406
72.23218536376953
56.855873107910156
49.23161315917969
56.44578170776367
48.19237518310547
48.225120544433594
48.26972198486328
52.93932342529297
69.0179672241211
90.00399017333984
112.00157928466797
131.30706787109375
144.93218994140625
151.0124053955078
148.9703369140625
139.48973083496094
124.31291961669922
105.81277465820312
86.83838653564453
69.44648742675781
56.065223693847656
47.952972412109375
45.55171585083008
71.04969787597656
48.76194763183594
48.76169967651367
48.74824142456055
48.71173858642578
48.65471267700195
48.58619689941406
48.516963958740234
48.46153259277344
48.430477142333984
48.43351745605469
48.47761917114258
48.39350128173828
48.3326416015625
48.30207061767578
48.27370071411133
48.253318786621094
48.23805236816406
48.228775024

100%|████████████████████████████████████| 10000/10000 [01:19<00:00, 126.56it/s]

108.00315856933594
98.55258178710938
118.03418731689453
158.1904754638672
207.6716766357422
254.65635681152344
289.22406005859375
SparseEDAA took 79.06s


In [ ]:
plt.plot(E0)

In [ ]:
plt.plot(B0)

In [ ]:
# Select a criterion to perform alignment
criterion = MSE()

# Select an aligner to perform alignment
A_true = A_true_np.reshape((EE.shape[1],nr1*nc1))#[1:p+1,:]
#aligner = MunkresAbundancesAligner2(A=A_true, criterion=criterion)

#A1 = aligner.fit_transform(A0)
#E1 = aligner.transform_endmembers(E0)

#
#x = A0.astype(np.float32).reshape((EE.shape[1],nr1*nc1))
SRE = 20*np.log10(LA.norm(A_true,'fro')/LA.norm((A_true - B0@A0),'fro'))

print ('SRE: %f' % (SRE))





In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=p, figsize=(12, 4))
A_reshape = A_true[1:p+1,:].reshape((p, nr1, nc1))
for kk in range(p):
    curr_ax = ax[kk]
    mappable = curr_ax.imshow(A_reshape[kk], vmin=0.0, vmax=1.0)
    fig.colorbar(mappable, ax=curr_ax, shrink=0.5)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=p, figsize=(12, 4))
A2 = (B0@A0)[1:p+1,:].reshape((p, nr1, nc1))
for kk in range(p):
    curr_ax = ax[kk]
    mappable = curr_ax.imshow(A2[kk], vmin=0.0, vmax=1.0)
    fig.colorbar(mappable, ax=curr_ax, shrink=0.5)